# Interagindo visualmente com dados

Analisar visualmente um conjunto de dados é uma tarefa importante no processo de ciência de dados. Até aqui, vimos como fazer isso usando gráficos estáticos produzidos pelas bibliotecas `matplotlib` e `seaborn`. Neste notebook, vamos explorar a biblioteca de visualição interativa de dados [Plotly](https://plot.ly/python/), uma biblioteca compatível com diferentes linguagens de programação. Para o ecossistema Python, podemos usar o módulo `plotly.express`, criado para facilitar a produção de visualizações interativas.

In [0]:
import pandas as pd
import plotly.express as px

Para esta análise, vamos utilizar [uma base de dados aberta sobre preços de combustíveis](https://http://dados.gov.br/dataset/infopreco), disponibilizada pelos postos no sítio da ANP. No código abaixo, vamos informar ao Pandas que trate a característica `DATA CADASTRO` como uma data:

In [0]:
preços = pd.read_csv('http://www.anp.gov.br/images/infopreco/infopreco.csv', encoding='latin-1', sep=';', decimal=',', parse_dates=['DATA CADASTRO'])
preços.head()

## Validando os dados

Vamos começar nossa análise validando os dados presentes na base. Isto é particularmente importante em bases de dados abertos governamentais, que muitas vezes não contam com boa curadoria de dados.

In [0]:
preços.isnull().sum()

In [0]:
preço_produto = preços.pivot_table(index="PRODUTO", values="VALOR VENDA")
preço_produto.head()

Os dados faltantes se limitam a bairro e complemento, que não serão foco da nossa análise. Em compensação, a análise do preço médio dos combustíveis listados indica valores consideravelmente elevados. Isto é um indicativo da presença de valores incorretos, que podemos verificar analisando a distribuição dos dados. Para interagir com o gráfico, passe o cursor do mouse sobre o gráfico e explore as opções no canto superior direito:

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

Note que o gráfico produzido pelo Plotly traz um número de ferramentas que podemos usar para aprofundar nossa investigação. Vamos destacar alguns deles:

* Ao posicionar o mouse sobre um boxplot, vemos suas informação.
* Ao posicionar o mouse sobre um outlier, podemos ver seu valor.
* Podemos dar zoom sobre as partes do gráfico que mais nos interessem.
* Podemos salvar o gráfico diretamente.

Analisando especificamente os dados da base em questão, os valores inválidos aparentam ter sido informados sem casas decimais. Vamos verificar quantos valores estão acima de 10 reais:

In [0]:
preços.query("`VALOR VENDA` > 10")

Por serem poucas observações, podemos removê-las sem prejuízo à base:

In [0]:
preços = preços.query("`VALOR VENDA` <= 10")

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

Curiosamente, agora vemos que também há outliers abaixo dos boxplots. Vamos investigar esses casos:

In [0]:
preços.query("`VALOR VENDA` <= 1")

Novamente, parece seguro remover estes casos:

In [0]:
preços = preços.query("`VALOR VENDA` > 1")

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

## Preço médio do combustível

Agora que limpamos os dados, vamos visualizar o preço médio por tipo de combustível:

In [0]:
preço_produto = preços.pivot_table(index="PRODUTO", values="VALOR VENDA")
preço_produto.head()

In [0]:
px.bar(preço_produto, x=preço_produto.index, y="VALOR VENDA", title='Preço médio por tipo de combustível')

Botemos refinar nossa análise analisando o preço da gasolina comum por estado:

In [0]:
preço_gasolina = preços.query("PRODUTO == 'Gasolina C Comum'")
gasolina_por_estado = preço_gasolina.pivot_table(index="UF", values='VALOR VENDA')
gasolina_por_estado.head()

Para gerar este gráfico a partir da tabela dinâmica acima, vamos informar que os índices das observações devem ser usados como valores para o eixo x:

In [0]:
px.bar(gasolina_por_estado, x=gasolina_por_estado.index, y="VALOR VENDA", title = 'Preço médio da Gasolina comum por UF')

Podemos expandir essa análise para incluir todos os produtos considerados. Para isto, vamos usar um histograma, informando com o parâmetro `histfunc="avg"` que estamos interessados no valor médio. Note que, apesar de ser um gráfico com muitas informações, é possível selecionar quais produtos observar clicando na legenda mostrada no lado direito.

In [0]:
px.histogram(preços, x="UF", y="VALOR VENDA", color="PRODUTO", histfunc="avg",
             barmode="group", title='Distribuição de preços por combustível')

## Analisando a evolução do preço dos combustíveis

Os dados disponíveis na base que baixamos são referentes a vários meses distintos. Chamamos este tipo de dado de uma **série temporal**, ou série histórica. Podemos visualizar a evolução destas séries usando gráficos de linhas. Para isso, precisamos inicialmente gerar a série para o valor médio de cada produto por mês.

O primeiro passo é produzir uma característica contendo apenas os dados de ano e mês, para que tenhamos dados suficientes para uma agregação. Fazemos isso usando o método `.dt.to_period("M")` que características reconhecidas como data apresentam:

In [0]:
preços['MES'] = preços['DATA CADASTRO'].dt.to_period('M').astype(str)
preços.head()

Um detalhe técnico do código acima é que o método `.dt.to_period("M")` gera um objeto do tipo `Period` atualmente incompatível com o gráfico de linhas da biblioteca Plotly. Por isso, pedimos para que o Pandas trate esta coluna como um texto usando o método `astype(str)`.

Agora que temos informação para cada mês, podemos gerar uma tabela dinâmica para visualizar a média por mês e produto:

In [0]:
preços_mês = preços.pivot_table(index="MES", columns="PRODUTO", values="VALOR VENDA")
preços_mês

Note que o `DataFrame`a acima contém muitos valores faltantes para o GNV. Assim, vamos descartar dados referentes a este produto:

In [0]:
preços_mês = preços_mês.drop("GNV", axis=1)
preços_mês.head()

Apesar de útil para exploração de dados, o formato da tabela dinâmica acima não é adequado para a produção de um gráfico de linhas com a biblioteca Plotly. O código abaixo converte o formato *wide* acima em um formato *longo*, usando os métodos `stack` e `reset_index`:

In [0]:
preços_mês = preços_mês.stack().reset_index(name="VALOR VENDA")
preços_mês.head()

Agora podemos investigar a evolução dos preços médios mensais de cada produto ao longo do período abrangido pela base de dados. Assim como no caso do histograma, podemos selecionar as séries que desejamos analisar interagindo com a legenda do gráfico:

In [0]:
px.line(preços_mês, x="MES", y="VALOR VENDA", color="PRODUTO",
        title='Evolução de preços por mês de observação (média do mês)')